# 蒸散发之作物需水估计

前面的FAO56主要对ET相关计算做了说明，接下来看看灌溉需水方面的计算。本文主要参考以下资料，并对其中内容做简单翻译记录。

- [usbr/et-demands](https://github.com/usbr/et-demands)
- [Historical and Future Irrigation Water Requirements for Select Reclamation Project Areas](https://www.usbr.gov/watersmart/baseline/docs/historicalandfutureirrigationwaterrequirements.pdf)
- [Huntington et al. (2015) West-Wide Climate Risk Assessments: Irrigation Demand and Reservoir Evaporation Projections](https://www.usbr.gov/watersmart/baseline/docs/irrigationdemand/irrigationdemands.pdf) 
- [Allen and Robison (2009)](http://www.kimberly.uidaho.edu/ETIdaho/)

以前想要在大范围内根据climate projections模拟计算未来的$ET_o$,$ET_c$,NIWR 的时间序列数据是基本不可能的，现在随着开源数据、代码的发展，这些都已经变得可能，因此，美国农垦局开发了相应的软件，即现在看到的ET-Demands，来估计historical和future的reference ET， crop ET和NIWR，也为考虑大气二氧化碳增加对蒸腾减少的影响做一些工作。

## Introduction

蒸散发是水循环的重要部分，也是农作物net irrigation water requirements（NIWR ）的主要驱动因素。量化ET和NIWR对区域农作物管理，流域水平衡估计都有很重要的意义。

另外，气候变化下的NIWR也是未知的，而这对长期的水资源规划管理是具有重要意义的。美国的安全水法案section 9503 授权了 美国农垦局在“major Reclamation river basins” 的气候变化对水和环境资源的风险。所以提供一个科学的能被广泛接受的估计历史和未来irrigation water demand 的方法是必要的。

## Previous work

基于Huntington et al.(2015) 的工作，他们使用开发的ET Demands工具（应该就是github上这一套）估计了crop ET和NIWR，不过那时候还不能计算各种climate projections下的情况，现在Python版本的开发等使得其可行。

建立在ASCE和FAO开发的灌溉需水模型基础上的ET Demands model很适合regional scale上的应用，已在很多地区得到应用。其增强了FAO-56的reference ET和dual crop coefficient approach：

- 模型基于FAO-56 reference ET和dual crop coefficient
- 广泛应用于美国西部的主要流域
- 考虑non-growing season soil moisture accumulation 和 变化的growing season lengths，灵活模拟future irrigation water demands
- 应用于West Wide Climate Risk Assessment（WWCRA） 研究

## Objectives

这个报告的主要目的是开发，应用开源crop ET 软件（ET Demands），帮助满足农垦局开发和应用ET 软件来估计历史climate和未来气候场景下的crop ET和NIWR 的需求。

## Approach

此研究实现了Huntington et al. (2015) ET Demands model的Python版本，并开发了针对多农年代表地区和历史未来气候的年的crop ET和NIWR估计。分析包括开发crop ET和NIWR 估计，结合World Climate Research Programme Coupled Model Intercomparison Project5 (WCRP CMIP5)  的气候projections。

## Methods

接下来记录一些细节。包括climate data， ET Demands 方法， 在历史和未来气候下的应用及结果。只是一些大致的过程，具体的细节可以更多参考Huntington et al.(2015) ，后面再补充。

### Reference ET

该值的计算还是基于FAO56或者ASCE-PM的计算$ET_o$的方式。然后乘上$K_c$得到$ET_c$。然后融合到ET Demands model。

### Climate Data

历史气候数据集从University of Idaho的METDATA 数据集获取，其是 gridded data，包括daily maximum and minimum air temperature, solar radiation, humidity, 和 wind speed。融合了North American Data Assimilation System(NLDAS)和the Parameter Regression on Independent Slopes Model (PRISM)。Point locations，又称为 Metnodes, 代表农业位置，是基于HUC8 区域 和 USDA cropland data layer (CDL) information的，用来提取历史和未来气候条件下的location的时间序列数据。

ET Demands model 在每个Metnode 上运行，使用从METADATA获取的1979-2015年的历史气候数据来计算 基于ASCE-PM 的$ET_o$，并且和基于天气站计算的$ET_o$比较。因为METADATA 没有考虑农业的evaporative cooling effect， 所以推求的$ET_o$会有biased high。为了去除偏差，$ET_o$计算使用了从18个农业天气站的数据，并计算了站点推求的$ET_o$和METADATA计算的$ET_o$的月尺度上的比值。然后用这个比例来消除这个偏差。

用Transient climate impacts methods involve 将time series climate projections转换为time series projections of weather inputs for impacts modeling。然后the Multivariate Adaptive Constructed Analogs (MACA) dataset 用到ASCE Standardized Penman-Monteith equation 来计算$ET_o$。MACA approach 包括校正映射daily General Circulation Model (GCM) data 到 网格数据的偏差。暂时不特别关注climate projection，所以暂略这部分。

### ET Demands Model Overview

ET Demands model 的核心是FAO-56 dual crop coefficient model。ASCE-PM-based grass reference ET0 用来作为baseline。每个Metnode上的Actual ET for a range of crop types 使用FAO-56 dual crop coefficient approach 计算：
$$ET_c=(K_sK_{cb}+K_e)ET_o$$
其中，

- $ET_o$ is ASCE-PM grass reference ET
- $K_{cb}$ is the basal crop coefficient
- $K_e$ is a coefficient representing evaporation from the soil surface

$K_{cb}$和 $K_e$是dimensionless的，变化从0到1.4。日$K_{cb}$值随season变化，这通常参考the crop coefficient curve。The stress coefficient ($K_s$) 从0到1，1表示没有water stress。ET Demands中一个daily soil water balance 用来模拟effective root zone以计算$K_s$。当为灌溉作物计算$ET_c$和 NIWR时，$K_s$通常为1，当降水较少时也会小于1。ET Demands 估计了冬天的$K_s$，对于休眠期的齐备mulch和grass，休眠期是没有灌溉的，所以冬天$K_s$小于1.第二个daily soil water balance for用来估算e upper 0.1 m of soil的$K_e$。这里假定 upper 0.1 m zone是唯一为土壤表面蒸发供水的部分。

NIWR 估算由 $ET_c$ 减去 保留在 root zone中的降水得到，即$P_{rz}$。保存在root zone中的Precipitation就是渗入土壤并保留在root zone用于蒸散发的消耗的那部分的gross reported precipitation。虽然$P_{rz}$包括了蒸发的降水，不过$ET_c$也包括降水的蒸发，因此 $ET_c - P_{rz}$ 可以代表the net irrigation water requirement，而不是让$ET_c$减去$P_{rz}$ 中用于蒸腾的部分代表。$P_{rz}$计算由$P-Runoff-D_{percp}$得到，其中P是gross reported precipitation，Runoff是模拟的surface runoff，$D_{percp}$是深度下渗到最大root zone下的那部分降水。更多细节稍后在Huntington et al. (2015)中详细看。

### Soils Data

ET Demands需要的Soil attributes从NRCS State Soil Geographic (STATSGO) database获取。STATSGO是一个空间土壤GIS数据库，包括了soilds物理特征属性，能用来估计土壤持水和渗透参数，并用于ET Demands model的dual soil and root zone water balance and runoff modules。STATSGO中关于available water capacity, and sand, silt, and clay fractions 的属性用于估计total evaporable water的空间分布和用于surface soil layer water balance的readily evaporable water以及用于root zone water balance的total available water and readily available water。这些参数都影响irrigation scheduling 的估计，soil的蒸发损失，deep percolation from root zones，antecedent soil moisture condition, and runoff from precipitation。Gridded soil attributes for available water capacity and sand, silt, and clay fractions 平均到 60-inch (150-cm) depths，然后和irrigated crop land areas 求交集, 并在空间上平均到每个 HUC8。

### Crop Coefficients

basal crop coefficient（$K_{cb}$）曲线来自Allen and Robison 并应用到应用到 ET Demands。Allen-Robison curves 是Wright的lysimeter-based $K_{cb}$ 曲线的继承，并且是基于taller alfalfa reference ET ($ET_r$)的。基于以下内容定义了不同的$K_{cb}$ 曲线：

1. 从 planting或者greenup到effective full cover的 ) normalized cumulative growing-degree-days，这个比例会持续到 the cropping period结束; 
2. 从 planting 到 effective full cover的时间百分比, 这个比例会持续到 the cropping period结束; 
3. 从 planting 到 effective full cover，然后再到之后一定天数（种植结束前）的时间百分比 

这些$K_{cb}$方法允许时间插值和特定crop的$K_{cb}$曲线作为cumulative growing degree days (CGDD) 和 temperature dependent planting or greenup estimates（比如，30 day moving average air temperature ($T_{30}$)）的函数，而不是特定的恒定的日历天。CGDD已经再很多研究中被用于定义planting and greenup times，crop coefficient development, scaling of development periods, 以及transferring $K_{cb}$ curves among regions。决定Kcb curves的planting，greenup，effective full cover和harvest stages的crop stage的参数的率定由运行ET Demands model来决定，该模型在每个Metnode中使用历史METDATA ihou时间序列，为每种crop确定最优的$T_{30}$，时间和到effective full cover 或 termination/harvest 的CGDD来拟合观测到的planting, greenup, 以及 development dates。当模拟的crop stages和每个流域子流域上记录的日期比较匹配时就得到最优值。

关于未来climate projections下的模拟还有未来二氧化碳影响的更多细节，这里暂不赘述了。

### ET Demands Model Application

ET Demands 模型用历史METDATA 时间序列来率定 $K_{cb}$ 曲线 和校正 $ET_o$的偏差。然后用来运行40个不同的MACA climate projection 1950-2100的时间序列来推求baseline（1950-2005）和projected（2006-2100）的各个metnode的年均温度，solar radiation，humidity， wind speed和totel precipitation，以及$ET_o$,$ET_c$和NIER。

Python代码模拟了每个$ET_o$，growing-season 和 non-growing-season soil 以及 root zone water balance components ($K_{cb}, K_s, K_e$),irrigations, 和 $ET_c$ 还有 NIWR，都是在日时间尺度上。$ET_o$ 在每个Metnode上计算，包括所有相关的信息，比如soil信息，crop type，和crop acreage，并用来计算$ET_c$和NIWR rates for 每个ET cell（比如HUC8 area）。日$ET_c$和NIWR rates 以及每个ET cell的volumes都使用从CDL data for 2010推求的unique irrigated crop types 和 associated areas计算。计算每个 ET cell (i.e. HUC8 area) 的 Crop area加权的 annual $ET_c$ 以及 NIWR rates ($ET_c – P_e$) 作为总的 annual ETc 和 NIWR volume。

## West-Wide Climate Risk Assessments: Irrigation Demand and Reservoir Evaporation Projections 相关内容补充

主要关注其中和ET Demands计算相关的内容，并且我主要关注HUC8流域上的baseline NIWR 的计算。它主要结合了Penman-Monteith (PM) reference ET method 和 FAO56的 dual crop coefficient method。ASCE-EWRI 2005 也用了FAO56的PM公式来计算cool season的clipped grass的$ET_o$。

计算实际蒸散发时选择的是–dual crop coefficient approach。改方法能更好地估计NIWR。

ET Demands model首先计算HUC8流域地各个点站点的日$ET_o$。

irrigated lands的空间平均的土壤参数也作为ET Demands的输入。

然后日crop ET 作为 $K_{cb}$ 和 $K_e$ 以及 $K_s$的函数进行计算。一个$K_s$是给流域内所有crop type用的。

最后NIWR rate or depth由 $ET_c$ 减去 effective precipitation residing in the root zone ($P_{rz}$) 得到。NIWR同义于这些名词： irrigation demand, net consumptive use, and precipitation deficit。$P_{rz}$计算是以 daily precipitation (from the climate data set)，antecedent soil moisture prior to a precipitation event, deep percolation of precipitation, and precipitation runoff.函数的形式得到。Soil moisture is也是根据流域内农业面积的情况土壤类型加权的moisture-holding capacity的函数得到的。产流是根据NRCS curve number method 得到的。

灌溉模拟是当crop root zone moisture content 下降到 crop-specific maximum allowable depletion threshold 时发生的。Irrigation depths 指定为 fill the root zone by the difference between field capacity and the cumulative soil moisture depletion depth amount.

流域上的NIWR 和 $ET_c$ 是按照作物类型面积加权得到的。

接下来更具体地过程结合到代码里再看。